In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
# df = pd.read_table('Data_28_F29.txt', delimiter=',')
df = pd.read_csv('cotton_data_03.csv')

In [3]:
df.head()

,ID,FIELD,YLD03,ELE,SLOPE,CURV,PRO,PLAN,EC_SH,EC_DP,...,BAND2,BAND3,BAND4,VI03_505,VI03_529,VI03_606,VI03_622,VI03_724,VI03_825,VI03_926
0,3018,C29,1082,1080.06,0.0879,0.002694,-0.001178,0.001515,14.4,38.700001,...,84,123,82,-0.042553,-0.215094,-0.157303,-0.136364,-0.021505,0.128571,0.085366
1,3019,C29,1202,1080.08,0.0575,0.001403,-0.000473,0.000930,14.3,40.900002,...,91,126,85,-0.053763,-0.220974,-0.169399,-0.163842,-0.010870,0.144828,0.144578
2,3020,C29,1196,1080.10,0.0906,-0.004624,0.004098,-0.000527,13.9,36.900002,...,92,127,83,-0.057592,-0.219331,-0.170213,-0.184358,-0.016043,0.140940,0.111111
3,3021,C29,1120,1080.13,0.1291,-0.001335,0.001011,-0.000324,14.1,37.099998,...,89,127,84,-0.071429,-0.236364,-0.165775,-0.169399,0.005405,0.120000,0.135135
4,3022,C29,1119,1080.18,0.1515,0.002538,-0.001229,0.001309,14.8,36.099998,...,89,130,85,-0.066667,-0.226277,-0.183246,-0.169399,-0.005291,0.099338,0.069767


In [3]:
# columns = ['AVGYLD','ELE','SLOPE','CURV', 'PRO','PLAN','EC_SH','EC_DP','BAND1','BAND2','BAND3','BAND4',
#            'VI00_520','VI00_528','VI00_613','VI00_707','VI00_715','VI00_723','VI00_816','VI00_824','VI00_901','VI00_917',
#            'VI01_616','VI01_624','VI01_710','VI01_827','VI01_912','VI02_518','VI02_526','VI02_619',
#            'VI02_713','VI02_721','VI02_830','VI02_907','VI02_923','VI03_505','VI03_529','VI03_606',
#            'VI03_622','VI03_724','VI03_825','VI03_926']

In [4]:

df_use = df.loc[:, df.columns != 'ID']

df_use = df_use[df_use['FIELD'] == 'C28']
df_use = df_use.loc[:, df_use.columns != 'FIELD']

df_use.head()

,YLD03,ELE,SLOPE,CURV,PRO,PLAN,EC_SH,EC_DP,BAND1,BAND2,BAND3,BAND4,VI03_505,VI03_529,VI03_606,VI03_622,VI03_724,VI03_825,VI03_926
475,465,1080.48,0.3589,0.020781,-0.015728,0.005053,3.4,5.8,119,139,220,138,-0.203125,-0.246575,-0.220779,-0.192000,-0.113122,-0.098266,-0.090000
476,482,1080.40,0.3689,-0.016757,0.013218,-0.003540,4.8,6.7,123,139,229,138,-0.206349,-0.236364,-0.201878,-0.214876,-0.144144,-0.029940,-0.085714
477,725,1080.35,0.2471,-0.031006,0.018248,-0.012759,5.5,11.5,119,137,225,132,-0.196653,-0.247387,-0.144231,-0.198238,-0.167421,0.018634,-0.068627
478,801,1080.39,0.2076,-0.020763,0.004777,-0.015986,4.7,11.1,113,133,220,132,-0.217391,-0.243986,-0.185185,-0.192661,-0.192825,0.006135,-0.020202
479,717,1080.47,0.2321,-0.009406,0.014907,0.005502,3.3,8.6,121,140,229,141,-0.201550,-0.239726,-0.205357,-0.214008,-0.144186,-0.041916,-0.019231


In [5]:
res_final = df[['ID', 'FIELD']]
res_final.head()

,ID,FIELD
0,3018,C29
1,3019,C29
2,3020,C29
3,3021,C29
4,3022,C29


In [6]:
df_use.columns.values

array(['YLD03', 'ELE', 'SLOPE', 'CURV', 'PRO', 'PLAN', 'EC_SH', 'EC_DP',
       'BAND1', 'BAND2', 'BAND3', 'BAND4', 'VI03_505', 'VI03_529',
       'VI03_606', 'VI03_622', 'VI03_724', 'VI03_825', 'VI03_926'], dtype=object)

In [7]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
import scikitplot as skplt

def compute_score(clf, X, y, scoring='mean_squared_error'):
    xval = cross_val_score(clf, X, y, cv = 5, scoring=scoring)
    return np.mean(xval)

def plot_roc(x_test, y_test, model):
    predicted_probas = model.predict_proba(x_test)
    skplt.metrics.plot_roc(y_test, predicted_probas, figsize =(10,10))
    plt.show()
    
def yld_category(yld):
    if yld <= 603:
        return 1
    elif 603 < yld <= 766:
        return 2
    elif 766 < yld <= 882:
        return 3
    elif 882 < yld <= 956:
        return 4
    elif 956 < yld <= 1018:
        return 5
    elif 1018 < yld <= 1075:
        return 6
    elif 1075 < yld <= 1134:
        return 7
    elif 1134 < yld <= 1200:
        return 8
    elif 1200 < yld <= 1280:
        return 9
    elif yld > 1280:
        return 10

In [12]:
full = df_use
full['AVGYLD_Category'] = full['YLD03'].apply(lambda x: yld_category(x))

In [13]:
# sns.distplot(full['AVGYLD_Category'], kde=False, bins=10, hist_kws={'edgecolor':'white', 'alpha':1.0})

In [14]:
# sns.distplot(full['AVGYLD_Category'], kde=False, bins=10, hist_kws={"rwidth":1,'edgecolor':'white', 'alpha':1.0})

In [15]:
tmp = []

for i in range(1, 11):
    tmp.append(full[full['AVGYLD_Category'] == i])

In [16]:
X_train = []
X_test = []
y_train = []
y_test = []
X_cols = [col for col in full.columns if col not in ['YLD03', 'AVGYLD_Category']]
X = []
y = []


for i in range(10):
    X.append(tmp[i][X_cols])
    y.append(tmp[i]['YLD03'])
    X_train_part, X_test_part, y_train_part, y_test_part = train_test_split(tmp[i][X_cols], tmp[i]['YLD03'], test_size=.2,random_state=0)
    X_train.append(X_train_part)
    X_test.append(X_test_part)
    y_train.append(y_train_part)
    y_test.append(y_test_part)

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(full[X_cols], full['YLD03'], test_size=.2,random_state=0)


X.append(full[X_cols])
y.append(full['YLD03'])
X_train.append(X_train_full)
X_test.append(X_test_full)
y_train.append(y_train_full)
y_test.append(y_test_full)



## dt

In [17]:
warnings.filterwarnings('ignore')
dt_tune_run = True
dt_parameters = []

for i in range(11):
    print(i)
    defaut = {'criterion': 'mse', 'max_depth': None,
                  'max_leaf_nodes': None, 'min_impurity_split': 0.005, 'min_weight_fraction_leaf': 0.0,
              'presort': False, 'splitter': 'random'}
    dt_parameters.append(defaut)
    if dt_tune_run:
        parameter_grid = {
    #          'max_features':range(0,1,1),
            'random_state':range(0,10,1),
            'min_samples_leaf':range(1,10,1),
            'min_samples_split': range(2, 20, 1)
        }
        forest = DecisionTreeRegressor(criterion='mse', max_depth=None,
                                        max_leaf_nodes=None,
                                        min_impurity_split=0.005, min_weight_fraction_leaf=0.0,
                                        presort=False, splitter='random')

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        DecisionTreeModel = grid_search
        parameters = grid_search.best_params_
        dt_parameters[i]['min_samples_leaf'] = grid_search.best_params_['min_samples_leaf']
        dt_parameters[i]['min_samples_split'] = grid_search.best_params_['min_samples_split']
        dt_parameters[i]['random_state'] = grid_search.best_params_['random_state']
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

0
Best score: -4122.077802841352
Best parameters: {'min_samples_leaf': 8, 'min_samples_split': 19, 'random_state': 5}
1
Best score: -1862.5050492426503
Best parameters: {'min_samples_leaf': 3, 'min_samples_split': 12, 'random_state': 6}
2
Best score: -843.6624188255018
Best parameters: {'min_samples_leaf': 6, 'min_samples_split': 15, 'random_state': 5}
3
Best score: -363.3053088336783
Best parameters: {'min_samples_leaf': 3, 'min_samples_split': 9, 'random_state': 7}
4
Best score: -388.2093178991166
Best parameters: {'min_samples_leaf': 3, 'min_samples_split': 15, 'random_state': 2}
5
Best score: -256.38493347704895
Best parameters: {'min_samples_leaf': 6, 'min_samples_split': 2, 'random_state': 0}
6
Best score: -220.16709633588866
Best parameters: {'min_samples_leaf': 6, 'min_samples_split': 2, 'random_state': 1}
7
Best score: -326.20411265009164
Best parameters: {'min_samples_leaf': 6, 'min_samples_split': 2, 'random_state': 0}
8
Best score: -525.881111524407
Best parameters: {'min_s

In [18]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
dt_model = []

for i in range(11):
    tmp_model = DecisionTreeRegressor(**dt_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    dt_model.append(tmp_model)

for i in range(11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(dt_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: dt range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: dt range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: dt range - {} measure: rmse, score: {}'.format(i+1, tmp))

0
Model: dt range - 1, measure: mae, score: 52.45
Model: dt range - 1 measure: mse, score: 4133.4
Model: dt range - 1 measure: rmse, score: 64.29
1
Model: dt range - 2, measure: mae, score: 34.29
Model: dt range - 2 measure: mse, score: 1870.81
Model: dt range - 2 measure: rmse, score: 43.25
2
Model: dt range - 3, measure: mae, score: 24.03
Model: dt range - 3 measure: mse, score: 840.97
Model: dt range - 3 measure: rmse, score: 29.0
3
Model: dt range - 4, measure: mae, score: 15.49
Model: dt range - 4 measure: mse, score: 361.74
Model: dt range - 4 measure: rmse, score: 19.02
4
Model: dt range - 5, measure: mae, score: 16.48
Model: dt range - 5 measure: mse, score: 383.79
Model: dt range - 5 measure: rmse, score: 19.59
5
Model: dt range - 6, measure: mae, score: 14.0
Model: dt range - 6 measure: mse, score: 261.94
Model: dt range - 6 measure: rmse, score: 16.18
6
Model: dt range - 7, measure: mae, score: 12.11
Model: dt range - 7 measure: mse, score: 215.54
Model: dt range - 7 measure

In [19]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def me(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return max(np.abs(y_true - y_pred))

In [20]:

for i in range(11):
    print(i)
    y_pred = dt_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: dt range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: dt range - {}, measure: me, score: {}'.format(i+1, me_res))

0
Model: dt range - 1, measure: mape, score: 12.71
Model: dt range - 1, measure: me, score: 99.12
1
Model: dt range - 2, measure: mape, score: 5.55
Model: dt range - 2, measure: me, score: 117.0
2
Model: dt range - 3, measure: mape, score: 1.69
Model: dt range - 3, measure: me, score: 51.21
3
Model: dt range - 4, measure: mape, score: 2.48
Model: dt range - 4, measure: me, score: 46.4
4
Model: dt range - 5, measure: mape, score: 2.18
Model: dt range - 5, measure: me, score: 35.62
5
Model: dt range - 6, measure: mape, score: 1.72
Model: dt range - 6, measure: me, score: 37.83
6
Model: dt range - 7, measure: mape, score: 1.78
Model: dt range - 7, measure: me, score: 27.25
7
Model: dt range - 8, measure: mape, score: 1.32
Model: dt range - 8, measure: me, score: 32.1
8
Model: dt range - 9, measure: mape, score: 1.87
Model: dt range - 9, measure: me, score: 51.33
9
Model: dt range - 10, measure: mape, score: 5.72
Model: dt range - 10, measure: me, score: 215.0
10
Model: dt range - 11, meas

In [21]:
y_pred_final = dt_model[10].predict(full[X_cols])
dt_pred_df = pd.DataFrame({'predicted_dt':y_pred_final})
dt_pred_df.head()
res_final_after_dt = res_final.join(dt_pred_df)
res_final_after_dt.head()

,ID,FIELD,predicted_dt
0,3018,C29,687.833333
1,3019,C29,490.454545
2,3020,C29,699.857143
3,3021,C29,699.857143
4,3022,C29,699.857143


## svm

In [22]:
warnings.filterwarnings('ignore')
svm_tune_run = True
svm_parameters = []

for i in range(11):
    print(i)
    defaut = {'cache_size': 200, 'coef0': 0.0,
                  'degree': 3, 'kernel': 'rbf', 'max_iter': -1,
              'shrinking': True, 'tol': 0.001, 'verbose': False}
    svm_parameters.append(defaut)
    if svm_tune_run:
        parameter_grid = {
             'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
             'gamma' : [0.001, 0.01, 0.1, 1]
         }

        forest = SVR(cache_size=200, coef0=0.0,
                  degree=3, kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        svm_parameters[i]['C'] = grid_search.best_params_['C']
        svm_parameters[i]['gamma'] = grid_search.best_params_['gamma']

        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

0
Best score: -4895.809368518458
Best parameters: {'C': 10, 'gamma': 0.001}
1
Best score: -2123.1557567103905
Best parameters: {'C': 1000, 'gamma': 0.1}
2
Best score: -1105.8807905178721
Best parameters: {'C': 100, 'gamma': 1}
3
Best score: -400.8126174165464
Best parameters: {'C': 1, 'gamma': 1}
4
Best score: -452.58015487514143
Best parameters: {'C': 100, 'gamma': 1}
5
Best score: -283.2696939153054
Best parameters: {'C': 1, 'gamma': 0.01}
6
Best score: -166.96209564798752
Best parameters: {'C': 100, 'gamma': 0.01}
7
Best score: -365.41907106595005
Best parameters: {'C': 100, 'gamma': 0.1}
8
Best score: -576.0352996940444
Best parameters: {'C': 1, 'gamma': 0.001}
9
Best score: -4826.257613440442
Best parameters: {'C': 1000, 'gamma': 1}
10
Best score: -33588.460865526176
Best parameters: {'C': 1000, 'gamma': 0.001}


In [23]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
svm_model = []

for i in range(11):
    tmp_model = SVR(**svm_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    svm_model.append(tmp_model)

for i in range(11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(svm_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: svm range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: svm range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: svm range - {} measure: rmse, score: {}'.format(i+1, tmp))

0
Model: svm range - 1, measure: mae, score: 57.43
Model: svm range - 1 measure: mse, score: 4886.16
Model: svm range - 1 measure: rmse, score: 69.9
1
Model: svm range - 2, measure: mae, score: 38.87
Model: svm range - 2 measure: mse, score: 2123.17
Model: svm range - 2 measure: rmse, score: 46.08
2
Model: svm range - 3, measure: mae, score: 27.79
Model: svm range - 3 measure: mse, score: 1100.83
Model: svm range - 3 measure: rmse, score: 33.18
3
Model: svm range - 4, measure: mae, score: 16.93
Model: svm range - 4 measure: mse, score: 401.79
Model: svm range - 4 measure: rmse, score: 20.04
4
Model: svm range - 5, measure: mae, score: 18.84
Model: svm range - 5 measure: mse, score: 447.73
Model: svm range - 5 measure: rmse, score: 21.16
5
Model: svm range - 6, measure: mae, score: 14.45
Model: svm range - 6 measure: mse, score: 287.99
Model: svm range - 6 measure: rmse, score: 16.97
6
Model: svm range - 7, measure: mae, score: 9.82
Model: svm range - 7 measure: mse, score: 163.96
Model

In [24]:

for i in range(11):
    print(i)
    y_pred = svm_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: svm range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: svm range - {}, measure: me, score: {}'.format(i+1, me_res))

0
Model: svm range - 1, measure: mape, score: 11.94
Model: svm range - 1, measure: me, score: 124.61
1
Model: svm range - 2, measure: mape, score: 3.99
Model: svm range - 2, measure: me, score: 102.17
2
Model: svm range - 3, measure: mape, score: 2.6
Model: svm range - 3, measure: me, score: 42.67
3
Model: svm range - 4, measure: mape, score: 1.89
Model: svm range - 4, measure: me, score: 31.3
4
Model: svm range - 5, measure: mape, score: 1.64
Model: svm range - 5, measure: me, score: 31.3
5
Model: svm range - 6, measure: mape, score: 1.3
Model: svm range - 6, measure: me, score: 26.72
6
Model: svm range - 7, measure: mape, score: 1.74
Model: svm range - 7, measure: me, score: 35.33
7
Model: svm range - 8, measure: mape, score: 1.47
Model: svm range - 8, measure: me, score: 30.53
8
Model: svm range - 9, measure: mape, score: 1.64
Model: svm range - 9, measure: me, score: 36.75
9
Model: svm range - 10, measure: mape, score: 5.56
Model: svm range - 10, measure: me, score: 190.88
10
Model

In [25]:
y_pred_final = svm_model[10].predict(full[X_cols])
svm_pred_df = pd.DataFrame({'predicted_svm':y_pred_final})
svm_pred_df.head()
res_final_after_svm = res_final_after_dt.join(svm_pred_df)
res_final_after_svm.head()

,ID,FIELD,predicted_dt,predicted_svm
0,3018,C29,687.833333,645.972532
1,3019,C29,490.454545,595.557582
2,3020,C29,699.857143,724.900128
3,3021,C29,699.857143,774.948630
4,3022,C29,699.857143,716.900435


## random forest

In [26]:
warnings.filterwarnings('ignore')
rf_tune_run = True
rf_parameters = []

for i in range(11):
    print(i)
    defaut = {'criterion': 'mse', 'max_depth': None, 'max_leaf_nodes': None,
                      'min_impurity_split': 0.005, 'min_weight_fraction_leaf': 0.0}
    rf_parameters.append(defaut)
    if rf_tune_run:
        parameter_grid = {
    #          'max_features':range(0,1,1),
            'random_state':range(0,10,1),
            'min_samples_leaf':range(1,10,1),
            'min_samples_split': range(2, 20, 1)
        }

        forest = RandomForestRegressor(criterion='mse', max_depth=None,
                                    max_leaf_nodes=None,
                                    min_impurity_split=0.005, min_weight_fraction_leaf=0.0)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        rf_parameters[i]['random_state'] = grid_search.best_params_['random_state']
        rf_parameters[i]['min_samples_leaf'] = grid_search.best_params_['min_samples_leaf']
        rf_parameters[i]['min_samples_split'] = grid_search.best_params_['min_samples_split']

        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

0
Best score: -4534.070118084492
Best parameters: {'min_samples_leaf': 9, 'min_samples_split': 19, 'random_state': 6}
1
Best score: -1845.6683952955664
Best parameters: {'min_samples_leaf': 3, 'min_samples_split': 8, 'random_state': 1}
2
Best score: -997.4362771527813
Best parameters: {'min_samples_leaf': 9, 'min_samples_split': 19, 'random_state': 9}
3
Best score: -402.58182420592635
Best parameters: {'min_samples_leaf': 1, 'min_samples_split': 19, 'random_state': 9}
4
Best score: -427.257643859514
Best parameters: {'min_samples_leaf': 1, 'min_samples_split': 3, 'random_state': 8}
5
Best score: -256.0352806835811
Best parameters: {'min_samples_leaf': 1, 'min_samples_split': 12, 'random_state': 5}
6
Best score: -227.21892300153084
Best parameters: {'min_samples_leaf': 1, 'min_samples_split': 16, 'random_state': 0}
7
Best score: -359.8397494185152
Best parameters: {'min_samples_leaf': 6, 'min_samples_split': 2, 'random_state': 3}
8
Best score: -597.8433201440606
Best parameters: {'min_s

In [27]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
rf_model = []

for i in range(11):
    tmp_model = RandomForestRegressor(**rf_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    rf_model.append(tmp_model)

for i in range(11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(rf_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: rf range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: rf range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: rf range - {} measure: rmse, score: {}'.format(i+1, tmp))

0
Model: rf range - 1, measure: mae, score: 55.89
Model: rf range - 1 measure: mse, score: 4543.69
Model: rf range - 1 measure: rmse, score: 67.41
1
Model: rf range - 2, measure: mae, score: 36.36
Model: rf range - 2 measure: mse, score: 1851.87
Model: rf range - 2 measure: rmse, score: 43.03
2
Model: rf range - 3, measure: mae, score: 25.84
Model: rf range - 3 measure: mse, score: 994.67
Model: rf range - 3 measure: rmse, score: 31.54
3
Model: rf range - 4, measure: mae, score: 16.88
Model: rf range - 4 measure: mse, score: 404.12
Model: rf range - 4 measure: rmse, score: 20.1
4
Model: rf range - 5, measure: mae, score: 16.81
Model: rf range - 5 measure: mse, score: 419.88
Model: rf range - 5 measure: rmse, score: 20.49
5
Model: rf range - 6, measure: mae, score: 13.44
Model: rf range - 6 measure: mse, score: 259.46
Model: rf range - 6 measure: rmse, score: 16.11
6
Model: rf range - 7, measure: mae, score: 12.37
Model: rf range - 7 measure: mse, score: 219.47
Model: rf range - 7 measu

In [28]:
for i in range(11):
    print(i)
    y_pred = rf_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: rf range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: rf range - {}, measure: me, score: {}'.format(i+1, me_res))

0
Model: rf range - 1, measure: mape, score: 10.63
Model: rf range - 1, measure: me, score: 115.4
1
Model: rf range - 2, measure: mape, score: 3.96
Model: rf range - 2, measure: me, score: 71.66
2
Model: rf range - 3, measure: mape, score: 2.07
Model: rf range - 3, measure: me, score: 37.49
3
Model: rf range - 4, measure: mape, score: 1.83
Model: rf range - 4, measure: me, score: 35.72
4
Model: rf range - 5, measure: mape, score: 2.01
Model: rf range - 5, measure: me, score: 41.38
5
Model: rf range - 6, measure: mape, score: 1.03
Model: rf range - 6, measure: me, score: 26.25
6
Model: rf range - 7, measure: mape, score: 1.86
Model: rf range - 7, measure: me, score: 29.44
7
Model: rf range - 8, measure: mape, score: 1.16
Model: rf range - 8, measure: me, score: 27.35
8
Model: rf range - 9, measure: mape, score: 1.8
Model: rf range - 9, measure: me, score: 48.56
9
Model: rf range - 10, measure: mape, score: 5.97
Model: rf range - 10, measure: me, score: 177.23
10
Model: rf range - 11, me

In [29]:
y_pred_final = rf_model[10].predict(full[X_cols])
rf_pred_df = pd.DataFrame({'predicted_rf':y_pred_final})
rf_pred_df.head()

,predicted_rf
0,463.257418
1,473.386786
2,714.349981
3,655.807838
4,577.689320


In [30]:
res_final_after_rf = res_final_after_svm.join(rf_pred_df)
res_final_after_rf.head()

,ID,FIELD,predicted_dt,predicted_svm,predicted_rf
0,3018,C29,687.833333,645.972532,463.257418
1,3019,C29,490.454545,595.557582,473.386786
2,3020,C29,699.857143,724.900128,714.349981
3,3021,C29,699.857143,774.948630,655.807838
4,3022,C29,699.857143,716.900435,577.689320


## xgboost

In [31]:
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')

xgb_tune_run = True
xgb_parameters = []

for i in range(11):
    print(i)
    defaut = {'subsample': 0.8, 'colsample_bytree': 1.0, 'max_depth': 2, 'gamma': 0.3, 'min_child_weight': 4}
    xgb_parameters.append(defaut)
    if xgb_tune_run:
        parameter_grid = {
                    'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)], 
                    'subsample':[i/10.0 for i in range(6,11)],
                    'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]
        }

        forest = XGBRegressor(nthread=-1)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        xgb_parameters[i]['min_child_weight'] = grid_search.best_params_['min_child_weight']
        xgb_parameters[i]['gamma'] = grid_search.best_params_['gamma']
        xgb_parameters[i]['subsample'] = grid_search.best_params_['subsample']
        xgb_parameters[i]['colsample_bytree'] = grid_search.best_params_['colsample_bytree']
        xgb_parameters[i]['max_depth'] = grid_search.best_params_['max_depth']
        
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

0
Best score: -6033.923812662882
Best parameters: {'colsample_bytree': 0.7, 'gamma': 0.3, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.7}
1
Best score: -2015.2241437288098
Best parameters: {'colsample_bytree': 0.7, 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.8}
2
Best score: -1224.5534006026846
Best parameters: {'colsample_bytree': 0.6, 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 4, 'subsample': 0.6}
3
Best score: -481.4164361881659
Best parameters: {'colsample_bytree': 0.6, 'gamma': 0.3, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.7}
4
Best score: -503.47705022736704
Best parameters: {'colsample_bytree': 0.7, 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 1.0}
5
Best score: -266.4489129373143
Best parameters: {'colsample_bytree': 0.6, 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 1.0}
6
Best score: -290.3978840368135
Best parameters: {'colsample_bytree': 0.9, 'gamma': 0.3, 'max_depth': 3, 'min_chil

In [32]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
xgb_model = []

for i in range(11):
    tmp_model = XGBRegressor(**xgb_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    xgb_model.append(tmp_model)

for i in range(11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(xgb_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: xbg range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: xbg range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: xbg range - {} measure: rmse, score: {}'.format(i+1, tmp))

0
Model: xbg range - 1, measure: mae, score: 63.18
Model: xbg range - 1 measure: mse, score: 6006.75
Model: xbg range - 1 measure: rmse, score: 77.5
1
Model: xbg range - 2, measure: mae, score: 37.21
Model: xbg range - 2 measure: mse, score: 2026.02
Model: xbg range - 2 measure: rmse, score: 45.01
2
Model: xbg range - 3, measure: mae, score: 28.37
Model: xbg range - 3 measure: mse, score: 1227.67
Model: xbg range - 3 measure: rmse, score: 35.04
3
Model: xbg range - 4, measure: mae, score: 18.55
Model: xbg range - 4 measure: mse, score: 483.7
Model: xbg range - 4 measure: rmse, score: 21.99
4
Model: xbg range - 5, measure: mae, score: 18.84
Model: xbg range - 5 measure: mse, score: 498.83
Model: xbg range - 5 measure: rmse, score: 22.33
5
Model: xbg range - 6, measure: mae, score: 14.61
Model: xbg range - 6 measure: mse, score: 267.24
Model: xbg range - 6 measure: rmse, score: 16.35
6
Model: xbg range - 7, measure: mae, score: 14.4
Model: xbg range - 7 measure: mse, score: 279.72
Model:

In [33]:
for i in range(11):
    print(i)
    y_pred = xgb_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: xgb range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: xgb range - {}, measure: me, score: {}'.format(i+1, me_res))

0
Model: xgb range - 1, measure: mape, score: 10.86
Model: xgb range - 1, measure: me, score: 150.02
1
Model: xgb range - 2, measure: mape, score: 4.04
Model: xgb range - 2, measure: me, score: 86.01
2
Model: xgb range - 3, measure: mape, score: 2.14
Model: xgb range - 3, measure: me, score: 46.72
3
Model: xgb range - 4, measure: mape, score: 1.95
Model: xgb range - 4, measure: me, score: 40.62
4
Model: xgb range - 5, measure: mape, score: 2.27
Model: xgb range - 5, measure: me, score: 45.51
5
Model: xgb range - 6, measure: mape, score: 0.85
Model: xgb range - 6, measure: me, score: 27.73
6
Model: xgb range - 7, measure: mape, score: 1.87
Model: xgb range - 7, measure: me, score: 41.85
7
Model: xgb range - 8, measure: mape, score: 1.42
Model: xgb range - 8, measure: me, score: 32.0
8
Model: xgb range - 9, measure: mape, score: 2.22
Model: xgb range - 9, measure: me, score: 54.01
9
Model: xgb range - 10, measure: mape, score: 6.36
Model: xgb range - 10, measure: me, score: 198.18
10
Mod

In [34]:
y_pred_final = xgb_model[10].predict(full[X_cols])
xgb_pred_df = pd.DataFrame({'predicted_xgb':y_pred_final})
xgb_pred_df.head()

,predicted_xgb
0,487.618195
1,492.002502
2,701.095154
3,717.655884
4,682.362366


In [35]:
res_final_after_xgb = res_final_after_rf.join(xgb_pred_df)
res_final_after_xgb.head()

,ID,FIELD,predicted_dt,predicted_svm,predicted_rf,predicted_xgb
0,3018,C29,687.833333,645.972532,463.257418,487.618195
1,3019,C29,490.454545,595.557582,473.386786,492.002502
2,3020,C29,699.857143,724.900128,714.349981,701.095154
3,3021,C29,699.857143,774.948630,655.807838,717.655884
4,3022,C29,699.857143,716.900435,577.689320,682.362366


## LinearRegression

In [36]:
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

lr_tune_run = True
lr_parameters = []

for i in range(11):
    print(i)
    lr_parameters.append({})
    if lr_tune_run:
        parameter_grid = {
             'copy_X': [True, False],
             'normalize': [True, False]
         }

        forest = LinearRegression()

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        parameters = grid_search.best_params_
        lr_parameters[i]['copy_X'] = grid_search.best_params_['copy_X']
        lr_parameters[i]['normalize'] = grid_search.best_params_['normalize']
        
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

0
Best score: -11989.748278713058
Best parameters: {'copy_X': True, 'normalize': False}
1
Best score: -3290.328336297643
Best parameters: {'copy_X': True, 'normalize': False}
2
Best score: -2296.7012660821892
Best parameters: {'copy_X': True, 'normalize': True}
3
Best score: -1007.5084617930761
Best parameters: {'copy_X': True, 'normalize': True}
4
Best score: -1128.0121454193634
Best parameters: {'copy_X': True, 'normalize': False}
5
Best score: -758.5442118242529
Best parameters: {'copy_X': True, 'normalize': True}
6
Best score: -4414.346315031054
Best parameters: {'copy_X': True, 'normalize': False}
7
Best score: -426.04416634298167
Best parameters: {'copy_X': True, 'normalize': False}
8
Best score: -2467.069116798168
Best parameters: {'copy_X': True, 'normalize': True}
9
Best score: -8370.40367577524
Best parameters: {'copy_X': True, 'normalize': True}
10
Best score: -17875.610687702236
Best parameters: {'copy_X': True, 'normalize': False}


In [37]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
lr_model = []

for i in range(11):
    tmp_model = LinearRegression(**lr_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    lr_model.append(tmp_model)

for i in range(11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(lr_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: lr range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: lr range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: lr range - {} measure: rmse, score: {}'.format(i+1, tmp))

0
Model: lr range - 1, measure: mae, score: 80.45
Model: lr range - 1 measure: mse, score: 12254.43
Model: lr range - 1 measure: rmse, score: 110.7
1
Model: lr range - 2, measure: mae, score: 47.23
Model: lr range - 2 measure: mse, score: 3312.4
Model: lr range - 2 measure: rmse, score: 57.55
2
Model: lr range - 3, measure: mae, score: 40.87
Model: lr range - 3 measure: mse, score: 2316.86
Model: lr range - 3 measure: rmse, score: 48.13
3
Model: lr range - 4, measure: mae, score: 24.95
Model: lr range - 4 measure: mse, score: 1017.18
Model: lr range - 4 measure: rmse, score: 31.89
4
Model: lr range - 5, measure: mae, score: 25.9
Model: lr range - 5 measure: mse, score: 1184.67
Model: lr range - 5 measure: rmse, score: 34.42
5
Model: lr range - 6, measure: mae, score: 21.52
Model: lr range - 6 measure: mse, score: 767.04
Model: lr range - 6 measure: rmse, score: 27.7
6
Model: lr range - 7, measure: mae, score: 48.36
Model: lr range - 7 measure: mse, score: 4558.69
Model: lr range - 7 me

In [38]:
for i in range(11):
    print(i)
    y_pred = lr_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: lr range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: lr range - {}, measure: me, score: {}'.format(i+1, me_res))

0
Model: lr range - 1, measure: mape, score: 13.67
Model: lr range - 1, measure: me, score: 255.69
1
Model: lr range - 2, measure: mape, score: 7.29
Model: lr range - 2, measure: me, score: 107.83
2
Model: lr range - 3, measure: mape, score: 3.39
Model: lr range - 3, measure: me, score: 69.37
3
Model: lr range - 4, measure: mape, score: 1.7
Model: lr range - 4, measure: me, score: 38.96
4
Model: lr range - 5, measure: mape, score: 2.88
Model: lr range - 5, measure: me, score: 48.77
5
Model: lr range - 6, measure: mape, score: 2.12
Model: lr range - 6, measure: me, score: 35.97
6
Model: lr range - 7, measure: mape, score: 2.5
Model: lr range - 7, measure: me, score: 62.21
7
Model: lr range - 8, measure: mape, score: 1.18
Model: lr range - 8, measure: me, score: 28.42
8
Model: lr range - 9, measure: mape, score: 2.01
Model: lr range - 9, measure: me, score: 48.53
9
Model: lr range - 10, measure: mape, score: 6.81
Model: lr range - 10, measure: me, score: 304.45
10
Model: lr range - 11, m

In [39]:
y_pred_final = lr_model[10].predict(full[X_cols])
lr_pred_df = pd.DataFrame({'predicted_lr':y_pred_final})
lr_pred_df.head()

,predicted_lr
0,490.277369
1,470.938963
2,581.526790
3,542.319390
4,560.387717


In [40]:
res_final_after_lr = res_final_after_xgb.join(lr_pred_df)
res_final_after_lr.head()

,ID,FIELD,predicted_dt,predicted_svm,predicted_rf,predicted_xgb,predicted_lr
0,3018,C29,687.833333,645.972532,463.257418,487.618195,490.277369
1,3019,C29,490.454545,595.557582,473.386786,492.002502,470.938963
2,3020,C29,699.857143,724.900128,714.349981,701.095154,581.526790
3,3021,C29,699.857143,774.948630,655.807838,717.655884,542.319390
4,3022,C29,699.857143,716.900435,577.689320,682.362366,560.387717


In [41]:
res_final_after_lr.to_csv('res_final_pred2.csv')